# Importance Scores Workflow

This notebook computes gradient-based importance scores for the following TF prediction tasks:
- CTCFL
- CEBPB
- E2F6
- Egr-1
- ELF1

The scores are saved as npy files and are stored in `/out/importance scores/DeepSea/`. The workflow is general and can be used for all prediction tasks in deepSEA.

In [2]:
# Select Environment(conda_kipoi-shared__env__kipoi-py3-keras2)
import os
import time
import h5py
import torch
import scipy.io
import numpy as np
import matplotlib.pyplot as plt

import kipoi
import kipoi_interpret
from kipoi_veff.utils.plot import seqlogo_heatmap
# Gradient-based methods
from kipoi_interpret.importance_scores.gradient import Gradient, GradientXInput
# Reference-based method
from kipoi_interpret.importance_scores.referencebased import DeepLift

In [2]:
### check if cuda is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cpu


In [3]:
### set the path
data_dir = '../../data/deepsea_train/'
result_dir = '../../out/importance scores/DeepSea/'

### Import DeepSEA model

In [4]:
deep_sea = kipoi.get_model("DeepSEA/predict")

Using downloaded and verified file: /home/ubuntu/.kipoi/models/DeepSEA/predict/downloaded/model_files/weights/89e640bf6bdbe1ff165f484d9796efc7


In [5]:
# print model architecture
deep_sea.model

Sequential(
  (0): ReCodeAlphabet()
  (1): ConcatenateRC()
  (2): Sequential(
    (0): Conv2d(4, 320, kernel_size=(1, 8), stride=(1, 1))
    (1): Threshold(threshold=0, value=1e-06)
    (2): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    (3): Dropout(p=0.2, inplace=False)
    (4): Conv2d(320, 480, kernel_size=(1, 8), stride=(1, 1))
    (5): Threshold(threshold=0, value=1e-06)
    (6): MaxPool2d(kernel_size=(1, 4), stride=(1, 4), padding=0, dilation=1, ceil_mode=False)
    (7): Dropout(p=0.2, inplace=False)
    (8): Conv2d(480, 960, kernel_size=(1, 8), stride=(1, 1))
    (9): Threshold(threshold=0, value=1e-06)
    (10): Dropout(p=0.5, inplace=False)
    (11): Lambda()
    (12): Sequential(
      (0): Lambda()
      (1): Linear(in_features=50880, out_features=925, bias=True)
    )
    (13): Threshold(threshold=0, value=1e-06)
    (14): Sequential(
      (0): Lambda()
      (1): Linear(in_features=925, out_features=919, bias=True)
    )
    (15):

### Load sample datasets

In [6]:
# sample data of size 10,000 and with corrected ordering 
with open(data_dir + 'X_test_sample.npy', 'rb') as f:
    X_test_sample = np.load(f)

In [7]:
print(X_test_sample.shape)

(10000, 4, 1, 1000)


### Compute importance scores for CEBPB, E2F6, Egr-1 and ELF1

In [13]:
tasks = {'CEBPB':337, 'E2F6':340, 'Egr-1':341, 'ELF1':342}

In [ ]:
for key, value in tasks.items():

    print(key, value)
    all_grxinp_scores = np.empty((10000, 1000, 4))
    all_gr_scores = np.empty((10000, 1000, 4))

    grxinp = GradientXInput(deep_sea, layer = '2.14.1', filter_idx = value) # value specifies TF index
    gr = Gradient(deep_sea, layer = '2.14.1', filter_idx = value) 

    batch_size = 1000
    for i in range(10):
        tic = time.time()
        start = i * batch_size
        end = start + batch_size

        grxinp_scores = grxinp.score(X_test_sample[start:end]) 
        grxinp_scores = grxinp_scores.squeeze().transpose((0,2,1))
        all_grxinp_scores[start:end, :, :] = grxinp_scores

        gr_scores = gr.score(X_test_sample[start:end])
        gr_scores = gr_scores.squeeze().transpose((0,2,1))
        row_means = np.mean(gr_scores, axis=1, keepdims=True) # mean normalize
        gr_scores = gr_scores - row_means
        all_gr_scores[start:end, :, :] = gr_scores

        toc = time.time()
        print(str(i) + ' iters completed', round(toc - tic), 'sec elapsed')

    np.save(result_dir + key + '_scores.npy', all_grxinp_scores)
    np.save(result_dir + key + '_hyp_scores.npy', all_gr_scores)

### Compute importance scores for CTCFL

In [ ]:
all_grxinp_scores = np.empty((10000, 1000, 4))
all_gr_scores = np.empty((10000, 1000, 4))

grxinp = GradientXInput(deep_sea, layer = '2.14.1', filter_idx = 339) # K562-CTCFL
gr = Gradient(deep_sea, layer = '2.14.1', filter_idx = 339) # K562-CTCFL

batch_size = 1000
for i in range(10):
    tic = time.time()
    start = i * batch_size
    end = start + batch_size
    
    grxinp_scores = grxinp.score(X_test_sample[start:end]) #n 4 1 1000
    grxinp_scores = grxinp_scores.squeeze().transpose((0,2,1))
    all_grxinp_scores[start:end, :, :] = grxinp_scores
    
    gr_scores = gr.score(X_test_sample[start:end])
    gr_scores = gr_scores.squeeze().transpose((0,2,1))
    row_means = np.mean(gr_scores, axis=1, keepdims=True)
    gr_scores = gr_scores - row_means
    all_gr_scores[start:end, :, :] = gr_scores
    
    toc = time.time()
    print(str(i) + ' iters completed', round(toc - tic), 'sec elapsed')

np.save(result_dir + 'CTCFL_scores.npy', all_grxinp_scores)
np.save(result_dir + 'CTCFL_hyp_scores.npy', all_gr_scores)
    
